# Train Momentum Detection Model

Train a model to detect momentum shifts from features.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import joblib
from pathlib import Path

# Load features
features = pd.read_parquet("../data/features/team_features.parquet")

# Create labels (momentum = 3+ win streak)
features["momentum_shift"] = (features.groupby(["match_id", "team"])["kills"].rolling(3).sum() >= 3).astype(int)

# Prepare features
X = features[["kills", "avg_kills_last_5"]].fillna(0)
y = features["momentum_shift"].fillna(0)

# Split
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model.fit(Xtr, ytr)

# Evaluate
y_pred = model.predict(Xte)
print(f"Accuracy: {accuracy_score(yte, y_pred):.3f}")
print(classification_report(yte, y_pred))

# Save model
Path("../app/models").mkdir(exist_ok=True)
joblib.dump(model, "../app/models/momentum.pkl")
print("Model saved to ../app/models/momentum.pkl")
